# Spark SQL

Demonstrate fundamental concepts in Spark SQL using the DataFrame API.

##### Objectives
1. Creating a table
1. Run a SQL query
1. Create a DataFrame from a table
1. Write the same query using DataFrame transformations
1. Trigger computation with DataFrame actions
1. Convert between DataFrames and SQL

##### Methods
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#spark-session-apis" target="_blank">SparkSession</a>: `sql`, `table`
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html" target="_blank">DataFrame</a>:
  - Transformations:  `select`, `where`, `orderBy`
  - Actions: `show`, `count`, `take`
  - Other methods: `printSchema`, `schema`, `createOrReplaceTempView`

## Multiple Interfaces
Spark SQL is a module for structured data processing with multiple interfaces.  

We can interact with Spark SQL in two ways:
1. Executing SQL queries
1. Working with the DataFrame API.

## 1. Create Database and Table

#### Databases in Spark  
Tables exist in Spark inside a database. So, We need to first talk about Databases before going to tables.  
If we don’t specify any database, Spark uses the default database.

We can see the list of available databases with ``listDatabases``:

This is how we can show the existing databases (with the DataFrame API):

In [0]:
#spark.catalog.listDatabases()

for database in spark.catalog.listDatabases():
    print(f"Name: {database.name}")
    print(f"Uri: {database.locationUri}")
    #print(f"Tables: {spark.catalog.listTables(database.name)}")
    print()

Name: default
Uri: s3://rbi-apex-at01-databricks-default-database-bucket/default

Name: dlt_test
Uri: s3://rbi-apex-at01-ho-apex-dev-gdwh-il-exploration-pipeline/gdwh_il_exploration_pipeline/dlt_test.db

Name: ds_academy_embedded_wave_3
Uri: s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3

Name: dsacademy
Uri: s3://rbi-apex-at01-ho-demo-dsacademy/dsacademy

Name: dsacademy_embedded_wave3_chiara
Uri: s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsacademy_embedded_wave3_chiara.db

Name: dsacademy_embedded_wave3_currentuser
Uri: s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsacademy_embedded_wave3_currentuser.db

Name: dsacademy_embedded_wave3_dmytro
Uri: s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_3/dsacademy_embedded_wave3_dmytro.db

Name: dsacademy_embedded_wave3_egezon
Uri: s3://rbi-apex-at01-ho-dsa-ds-academy-embedded-wave-3/ds_academy_embedded_wave_

In [0]:
spark.sql('show databases').show()

+--------------------+
|        databaseName|
+--------------------+
|             default|
|            dlt_test|
|ds_academy_embedd...|
|           dsacademy|
|dsacademy_embedde...|
|dsacademy_embedde...|
|dsacademy_embedde...|
|dsacademy_embedde...|
|dsacademy_embedde...|
|dsacademy_embedde...|
|    dsacademy_renato|
|gdwh_il_explorati...|
|           gdwh_test|
|          gdwhbranch|
|           ho_norkom|
|  nbo_microtxsc_rbko|
|            nbo_rbal|
|norkom_alertscori...|
|norkom_exploratio...|
| ofsa_exploration_ho|
+--------------------+
only showing top 20 rows



Using SQL:

In [0]:
%sql
show databases;

databaseName
default
dlt_test
ds_academy_embedded_wave_3
dsacademy
dsacademy_embedded_wave3_chiara
dsacademy_embedded_wave3_currentuser
dsacademy_embedded_wave3_dmytro
dsacademy_embedded_wave3_egezon
dsacademy_embedded_wave3_renato
dsacademy_embedded_wave3_wzhpoo


#### Creating Database 
*(with DataFrame API)*

Let us take a look at the current database:

In [0]:
spark.catalog.currentDatabase()

You will now create databases in your workspace.  
To name it correctly, **substitute "renato" by your name in the `username` variable.

In [0]:
## Put your name here
username = "renato"

dbutils.widgets.text("username", username)
spark.sql(f"CREATE DATABASE IF NOT EXISTS dsacademy_embedded_wave3_{username}")
spark.sql(f"USE dsacademy_embedded_wave3_{username}")
spark.conf.set("spark.sql.shuffle.partitions", 8)

spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

Test if the database was created (with DataFrame API):

In [0]:
#spark.catalog.listTables(f'dsacademy_embedded_wave3_{username}')
spark.sql(f'show tables from dsacademy_embedded_wave3_{username}').show()

#### Using SQL 

We need to creat a [widget](https://docs.databricks.com/notebooks/widgets.html) to allow for substituting the user in subsequent queries.  
Substitute **"renato"** by **"your_name"** in the **CREATE WIDGET TEXT username DEFAULT "renato"** command.

In [0]:
%sql
CREATE WIDGET TEXT username DEFAULT "renato";
SHOW TABLES IN dsacademy_embedded_wave3_$username;

#####Creating a table using SQL

We now create a table using artificial data generated by [generatedata](https://generatedata.com/generator).

In [0]:
%sql

CREATE OR REPLACE TABLE dsacademy_embedded_wave3_$username.testtable (
  id int NOT NULL,
  name varchar(255),
  email varchar(255),
  address varchar(255),
  postalZip varchar(15),
  region varchar(60),
  country varchar(100)
);

INSERT INTO dsacademy_embedded_wave3_$username.testtable 
  (id, name, email, address, postalZip, region, country)
VALUES
  (1, "Isabella Best","penatibus.et@yahoo.edu","339-2019 A Ave","233901","East Java","Singapore"),
  (2, "Ruby Clayton","non@icloud.com","648-3979 Suspendisse Av.","8898","North-East Region","South Korea"),
  (3, "Pandora Lyons","justo.proin@yahoo.ca","299-8784 A, Ave","9317","Cagayan Valley","South Korea"),
  (4, "George Figueroa","nec@icloud.org","P.O. Box 952, 7021 Nam St.","20343","Kherson oblast","Nigeria"),
  (5, "Preston Owens","a.odio@google.couk","915-551 Lorem Road","901457","KwaZulu-Natal","Colombia"),
  (6,"Mark Bullock","parturient.montes.nascetur@hotmail.com","Ap #431-9266 Eu, St.","2874 WP","Hòa Bình", "Japan"),
  (7,"Gavin Daniel","hendrerit@protonmail.net","P.O. Box 366, 573 Blandit Ave","1822-5149","Ilocos Region", "China"),
  (8,"Hayden Strickland","at@outlook.couk","192-9529 Velit. Avenue","11759","Styria", "Austria"),
  (9,"Walter Waller","a.tortor.nunc@yahoo.couk","P.O. Box 433, 9102 Parturient St.","3967","Kaduna", "Butan"),
  (10,"Erich Sears","eu.ultrices.sit@aol.edu","4372 Neque Street","47797","Maranhão", "Brazil"),
  (11,"Maryam Wilkins","neque.in@icloud.com","350-8918 Quis, St.","38935","Gauteng","Ukraine"),
  (12,"Elton Yates","donec.sollicitudin@outlook.couk","P.O. Box 914, 9443 Et Ave","51091","Araucanía","Netherlands"),
  (13,"Ira Bailey","mauris@aol.ca","5615 Ultricies Rd.","1852","Osun","China"),
  (14,"Allistair Fuller","ornare.tortor@protonmail.couk","Ap #394-5115 Neque Rd.","10460","North Region","Ireland"),
  (15,"Griffin Rowland","lorem@protonmail.ca","Ap #655-5364 Ligula. Rd.","12593","Heredia","Spain"),
  (16,"George Carter","orci.lacus@google.com","Ap #657-1977 Malesuada Avenue","53560","Khmelnytskyi oblast","Peru"),
  (17,"Lester Deleon","vivamus.non.lorem@google.org","P.O. Box 810, 4634 Tempor, Avenue","947671","Baden Württemberg","United States"),
  (18,"Elmo O'brien","feugiat@google.ca","Ap #248-5163 Mauris Street","4784","Vienna","China"),
  (19,"Joshua Stewart","proin.mi.aliquam@icloud.com","473-9769 Gravida. Street","36511","Drenthe","South Africa"),
  (20,"Jacqueline Joseph","a.malesuada.id@yahoo.net","Ap #837-4938 Aliquam Rd.","62-93","North Gyeongsang","Ukraine"),
  (21,"Uriah Shepard","ut.eros@hotmail.com","Ap #214-145 Malesuada Avenue","S5B 6J1","Zhōngnán","Norway"),
  (22,"Lucas Goodwin","blandit.at.nisi@icloud.net","911-994 Metus Rd.","5936","Khyber Pakhtoonkhwa","Brazil"),
  (23,"Baxter Mcgowan","congue.in@google.couk","Ap #655-8849 Nam Road","47-822","Vaupés","Ireland"),
  (24,"Xavier Forbes","cum.sociis@protonmail.org","285-9211 Ipsum Rd.","VV4 9JW","Riau Islands","United Kingdom"),
  (25,"Paul Shaffer","eu.tellus@outlook.couk","Ap #998-5591 Mauris Street","445634","Xīnán","Brazil"),
  (26,"Ignacia Mclean","nisi.a@yahoo.net","404-5251 Tellus, Ave","11607","Wiltshire","Turkey"),
  (27,"Levi Johnston","eget.nisi@icloud.net","685-5642 Semper Rd.","859819","Ternopil oblast","Colombia"),
  (28,"Velma Tran","consectetuer.rhoncus@yahoo.edu","758-2107 Tellus, Road","61905","Central Region","Norway"),
  (29,"Quin Campos","maecenas.mi@hotmail.net","Ap #333-7020 Sed Rd.","0629-6367","Abruzzo","Philippines"),
  (30,"Colleen Moses","vivamus.euismod@outlook.com","Ap #686-5389 Erat St.","64848-67112","North Island","United Kingdom"),
  (31,"Noah Albert","a.magna@icloud.couk","Ap #932-1166 Sagittis St.","776334","West Sulawesi","Vietnam"),
  (32,"Evan Avila","suspendisse.ac@google.couk","P.O. Box 612, 7585 Et Rd.","22877-63182","Vestfold og Telemark","Ukraine"),
  (33,"Nicholas Wooten","lorem.ac@protonmail.com","P.O. Box 913, 3734 Neque. Road","97754","Quindío","South Korea"),
  (34,"Mariam Hinton","elit@icloud.edu","125-5355 Est Rd.","11356","Drenthe","Austria"),
  (35,"Cullen Trujillo","non.bibendum.sed@hotmail.edu","998-3377 Velit. Street","8373","Western Cape","Pakistan"),
  (36,"Quinlan Hendricks","convallis@outlook.ca","Ap #787-3999 Libero. Rd.","368034","Maule","China"),
  (37,"Addison Mueller","in.felis.nulla@aol.edu","9680 Lacus Rd.","135380","Tasmania","Colombia"),
  (38,"Eagan Carroll","ipsum.porta.elit@protonmail.edu","660-9107 Aliquet St.","257716","West Region","Austria"),
  (39,"Luke Farley","eget.varius.ultrices@yahoo.couk","692-1967 Donec Avenue","47884","Boyacá","South Africa"),
  (40,"Brendan Dotson","per@icloud.ca","7325 Sagittis. Ave","30500","Puno","India"),
  (41,"Fletcher Morrison","dictum.proin.eget@protonmail.ca","Ap #640-4165 Arcu. Rd.","46829","Anambra","Chile"),
  (42,"Malik Moss","auctor@google.net","817-8978 Donec St.","7176","Guerrero","Ukraine"),
  (43,"Jermaine Hunt","dignissim.tempor@aol.net","185-9352 Dui. Ave","14-551","Western Visayas","Pakistan"),
  (44,"Bryar Michael","suscipit.est.ac@protonmail.net","472-1651 Ac Rd.","78314-76099","Styria","Pakistan"),
  (45,"Juliet Fernandez","odio.phasellus@google.couk","Ap #670-201 Ac Street","4644","Quảng Bình","Canada"),
  (46,"Ulysses Banks","mauris.eu.elit@hotmail.com","Ap #132-2097 Vestibulum, Road","263577","Vlaams-Brabant","Germany"),
  (47,"Gray Gibson","erat.neque.non@aol.ca","749-8689 Donec Road","292992","Vaupés","China"),
  (48,"Buffy Battle","ut.nulla@google.org","Ap #542-9101 Metus. Street","37025","Paraíba","Peru"),
  (49,"Price Brock","pharetra.sed.hendrerit@outlook.com","P.O. Box 680, 3103 Quis, Av.","667810","Luik","Mexico"),
  (50,"Murphy Leon","amet@icloud.com","948-5558 Vivamus Av.","48368","South Island","Netherlands"),
  (52,"Maxwell Whitfield","donec.dignissim@outlook.net","3496 Nibh Rd.","237245","Limousin","Russian Federation"),
  (53,"Jermaine Wyatt","integer@yahoo.ca","952-3916 Sed St.","48-564","Dolnośląskie","Vietnam"),
  (54,"Joelle Baird","cras.eget.nisi@yahoo.couk","259-5104 Ac Street","9933","Andalucía","Mexico"),
  (55,"Hoyt Mcdaniel","auctor.mauris@google.org","Ap #476-7147 Nulla Avenue","32354","East Region","Chile"),
  (56,"Bo Baker","tincidunt.nibh@outlook.edu","893-172 Non Street","5388","Xīnán","Nigeria"),
  (57,"Cherokee Chapman","sed.tortor@hotmail.couk","6472 In Street","45-76","Cartago","Costa Rica"),
  (58,"Leah Alford","eu.lacus@protonmail.ca","P.O. Box 449, 4616 Molestie Street","50174","Jönköpings län","Nigeria"),
  (59,"Cassandra Travis","quisque.porttitor@outlook.ca","Ap #210-5623 Et, Rd.","397392","Connacht","Costa Rica"),
  (60,"Deirdre Daugherty","dignissim@icloud.org","Ap #851-4729 Non Av.","13521","Cordillera Administrative Region","Chile"),
  (61,"Jordan Chen","rutrum@protonmail.org","241-109 Cum Ave","6900 XL","North Island","Colombia"),
  (62,"Kaitlin Taylor","feugiat.nec@google.ca","7654 Penatibus St.","203414","Meghalaya","Belgium"),
  (63,"Bethany Williams","ornare.fusce@aol.couk","695-2555 Cras Av.","5047 XA","North Island","New Zealand"),
  (64,"Veda Chandler","arcu.ac@outlook.ca","2137 Egestas. Road","67-62","Friesland","Belgium"),
  (65,"Mason Morgan","nisl@aol.net","Ap #241-2562 Tincidunt Rd.","77548","Zhōngnán","Italy"),
  (66,"Alana Griffin","eget.mollis@aol.net","Ap #660-3434 Varius Ave","881399","Burgenland","Peru"),
  (67,"Iliana Gregory","sagittis.semper.nam@hotmail.edu","275-2955 Blandit. Rd.","81822","Junín","Mexico"),
  (68,"Dolan Brady","cursus.luctus@aol.net","Ap #964-3929 Amet, Av.","53457-20818","Cartago","Philippines"),
  (69,"Echo Conley","amet.ante@icloud.ca","925-5694 Facilisis Street","434021","Noord Holland","Belgium"),
  (70,"Callum Wiley","semper@yahoo.ca","P.O. Box 563, 9298 Dolor St.","PE79 5TS","Kocaeli","Norway"),
  (71,"Winifred Rodgers","elit.nulla@google.net","264-1732 Justo. Avenue","441222","Maule","Mexico"),
  (72,"Curran Gilbert","risus.donec@icloud.edu","7648 Cum Avenue","35069","Manitoba","Philippines"),
  (73,"Jennifer Mckay","augue@yahoo.edu","979-8352 Egestas. Rd.","6736","Bắc Kạn","Germany"),
  (74,"Leslie Foley","libero@protonmail.net","Ap #981-9632 Interdum St.","52829-853","Aragón","Pakistan"),
  (75,"Phoebe Key","sed.dolor@yahoo.net","Ap #527-3178 Turpis. Avenue","6653","Navarra","South Korea"),
  (76,"Barry Prince","vehicula.risus@outlook.net","P.O. Box 925, 7842 Id Road","995123","Tiền Giang","Indonesia"),
  (77,"Kylynn Evans","mi.felis.adipiscing@protonmail.com","425 Ante Ave","283515","Swiętokrzyskie","Sweden"),
  (78,"Laurel Mckenzie","elit@icloud.net","Ap #806-7947 A St.","27-891","São Paulo","Russian Federation"),
  (79,"Roanna Howell","mauris.sagittis@aol.com","868-2211 Dolor, St.","85-938","Gävleborgs län","Indonesia"),
  (80,"Leo Hahn","quis.massa@outlook.couk","356-9688 Eleifend, St.","24735-627","Paraná","China"),
  (81,"Victoria Henry","ultricies.ligula@protonmail.couk","995-3711 Sit Ave","01198","Putumayo","Mexico"),
  (82,"Thor Donovan","sed@yahoo.org","541-4356 Velit Rd.","989431","Magallanes y Antártica Chilena","Indonesia"),
  (83,"Clementine Spencer","etiam.vestibulum@icloud.couk","Ap #502-7898 Curabitur St.","62435-52851","Denbighshire","Australia"),
  (84,"Fredericka Blake","tempor.arcu@hotmail.com","P.O. Box 201, 5087 Mauris Rd.","55923","Điện Biên","Poland"),
  (85,"Bruce Dunn","sed.sapien@yahoo.ca","Ap #444-6464 Faucibus Rd.","73716","Australian Capital Territory","Austria"),
  (86,"Henry Higgins","morbi.accumsan@icloud.com","P.O. Box 186, 2472 Integer St.","63461","Salzburg","Spain"),
  (87,"Brock Nunez","nec.tempus@google.edu","P.O. Box 809, 505 Dictum St.","66-996","Gävleborgs län","China"),
  (88,"Grady Horn","libero.at.auctor@hotmail.net","Ap #319-6785 Nisi Rd.","16682","San José","South Korea"),
  (89,"Jin Schultz","cursus@icloud.couk","9723 Cursus St.","164583","Hòa Bình","Germany"),
  (90,"Leo Jenkins","vulputate.eu.odio@aol.org","3775 Nonummy. St.","11113","Newfoundland and Labrador","Sweden"),
  (91,"Britanney Dejesus","volutpat.nulla.dignissim@icloud.org","Ap #185-5931 Morbi Ave","29986","Bayern","Ukraine"),
  (92,"Rahim Black","accumsan.interdum@aol.ca","P.O. Box 528, 4916 Nonummy St.","98-27","Zamboanga Peninsula","United Kingdom"),
  (93,"Ruth Lowery","lectus.pede.ultrices@hotmail.couk","Ap #735-2049 Tellus Ave","14231","Vichada","New Zealand"),
  (94,"Edan Clark","nulla.at@yahoo.net","P.O. Box 705, 8272 Lorem Rd.","4446","Coahuila","Ukraine"),
  (95,"Denise Bradshaw","tempus@yahoo.com","Ap #759-6875 Sapien, Rd.","15686","Ulster","Colombia"),
  (96,"Skyler Hamilton","egestas.a@hotmail.com","135-1715 Vulputate, St.","485617","Metropolitana de Santiago","Sweden"),
  (97,"Irma Evans","suspendisse.sagittis.nullam@outlook.net","5186 Vitae, St.","65364","Mpumalanga","Chile"),
  (98,"Wendy Aguilar","diam.vel@protonmail.couk","Ap #335-6690 Sit Street","08847","Connacht","Canada"),
  (99,"Kermit Taylor","hendrerit.donec.porttitor@google.couk","5657 Neque Street","8065","Lower Austria","Indonesia"),
  (100,"Keiko Buck","eu.neque@protonmail.ca","Ap #543-8063 Ridiculus Street","865263","Tamaulipas","India"),
  (101,"Whoopi Villarreal","arcu.curabitur.ut@aol.org","789-5908 Curabitur St.","313473","Vestfold og Telemark","Brazil"),
  (102,"Kylynn Goodwin","cras.pellentesque@icloud.net","P.O. Box 443, 642 Ut Avenue","844772","Davao Region","Costa Rica"),
  (103,"Yen Woodward","massa.mauris@aol.couk","794-1193 Sapien. Ave","02438","Saskatchewan","Pakistan"),
  (104,"Jermaine Gilliam","magnis@google.ca","8313 Eget, Ave","6689-8405","Araucanía","Italy"),
  (105,"Thomas Beasley","aliquam.auctor@google.net","661-1095 Sit Av.","03530","Western Australia","Brazil"),
  (106,"Ferris Wallace","sed.est@google.net","Ap #112-1359 Neque Road","4185","Oaxaca","South Africa"),
  (107,"Magee Glass","fermentum.metus.aenean@google.net","390-509 In Ave","74-56","South Island","India"),
  (108,"Kevyn Branch","nunc@hotmail.edu","3688 Orci. St.","81872","Kayseri","Russian Federation"),
  (109,"Alma Terrell","justo.sit.amet@google.org","238-679 Ante Street","3447","Antalya","Nigeria"),
  (110,"Barry Richmond","luctus@aol.net","Ap #317-2926 Molestie Av.","18015","North Chungcheong","Australia"),
  (111,"Mia Huffman","tincidunt@aol.net","3064 Et Avenue","51132","Vienna","Turkey");  

##### Check table creation

Using (with DataFrame API)

In [0]:
for database in spark.catalog.listDatabases():
  if "dsacademy_embedded_wave3" in database.name:
    print(f"Name: {database.name}")
    print(f"Uri: {database.locationUri}")
    print(f"Tables: {spark.catalog.listTables(database.name)}")
    print()

Using SQL:

In [0]:
%sql
USE dsacademy_embedded_wave3_$username;
SHOW TABLES;

You can inspect the result of the query as a dataframe (stored in the variable "_sqldf"):

In [0]:
_sqldf.show()

### 5. Query table and plot results

Below, you can experiment a bit more on how to query a table and then plot the results by selecting the plus sign `+` icon.

In [0]:
%sql
SELECT * 
FROM dsacademy_embedded_wave3_$username.testtable
LIMIT 30
-- WHERE name LIKE 'R%'
;

In [0]:
_sqldf.show()

**Executing SQL queries**

In [0]:
%sql
SELECT name, country
FROM dsacademy_embedded_wave3_$username.testtable
WHERE name LIKE 'P%'
ORDER BY country

**Queries with the DataFrame API**

We can also express Spark SQL queries using the DataFrame API.  
The following cell returns a DataFrame containing the same results as those retrieved above.

In [0]:
display(spark.table("testtable")
  .select("name", "country")
  .where("name LIKE 'P%'")
  .orderBy("country"))

We'll go explore more of the syntax of the DataFrame API.  
You can already see how this builder design pattern allows us to chain a sequence of operations, very similar to those we find in SQL.

## Query Execution
We can express the same query using any interface. The Spark SQL engine generates the same query plan used to optimize and execute on our Spark cluster.

![query execution engine](https://files.training.databricks.com/images/aspwd/spark_sql_query_execution_engine.png)

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Resilient Distributed Datasets (RDDs) are the low-level representation of datasets processed by a Spark cluster. In early versions of Spark, you had to write <a href="https://spark.apache.org/docs/latest/rdd-programming-guide.html" target="_blank">code manipulating RDDs directly</a>. In modern versions of Spark you should instead use the higher-level DataFrame APIs, which Spark automatically compiles into low-level RDD operations.

## Spark API Documentation

To learn how we work with DataFrames in Spark SQL, let's first look at the Spark API documentation.  
The main Spark [documentation](https://spark.apache.org/docs/latest/) page includes links to API docs and helpful guides for each version of Spark.  

The [Scala API](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/index.html) and [Python API](https://spark.apache.org/docs/latest/api/python/index.html) are most commonly used, and it's often helpful to reference the documentation for both languages.  
Scala docs tend to be more comprehensive, and Python docs tend to have more code examples.

#### Navigating Docs for the Spark SQL Module
Find the Spark SQL module by navigating to `org.apache.spark.sql` in the Scala API or `pyspark.sql` in the Python API.  
The first class we'll explore in this module is the `SparkSession` class. You can find this by entering "SparkSession" in the search bar.

## SparkSession
The `SparkSession` class is the single entry point to all functionality in Spark using the DataFrame API. 

In Databricks notebooks, the SparkSession is created for you automatically, stored in a variable called `spark`.

In [0]:
spark

Let's use the SparkSession method `table` to create a DataFrame from the `testtable` table.  
Let's save this in the variable `users`.

In [0]:
users = spark.table("testtable")
users.show()

Below are several additional methods we can use to create DataFrames. All of these can be found in the <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html" target="_blank">documentation</a> for `SparkSession`.

#### `SparkSession` Methods
| Method | Description |
| --- | --- |
| sql | Returns a DataFrame representing the result of the given query | 
| table | Returns the specified table as a DataFrame |
| read | Returns a DataFrameReader that can be used to read data in as a DataFrame |
| range | Create a DataFrame with a column containing elements in a range from start to end (exclusive) with step value and number of partitions |
| createDataFrame | Creates a DataFrame from a list of tuples, primarily used for testing |

Let's use a SparkSession method to run SQL.  
Remember that we defined the default database when we ran ``spark.sql(f"USE dsacademy_embedded_wave3_{username}")``

In [0]:
df_usersP = spark.sql("""
SELECT name, country
FROM testtable
WHERE name LIKE 'P%'
ORDER BY country
""")

display(df_usersP)

## DataFrames
A **DataFrame** is a distributed collection of data grouped into named columns.

Recall that expressing our query using methods in the DataFrame API results in a DataFrame.  
Let's store this in the variable `UsersP`.

In [0]:
usersP = (spark.table("testtable")
  .select("name", "country")
  .where("name LIKE 'P%'")
  .orderBy("country"))

We can use `display()` to output the results of a dataframe.

In [0]:
display(usersP)

The **schema** defines the column names and types of a dataframe.

You can access a dataframe's schema using the `schema` attribute.

In [0]:
usersP.schema

View a nicer output for this schema using the `printSchema()` method.

In [0]:
usersP.printSchema()

## Transformations
When we created `usersP`, we used a series of DataFrame transformation methods e.g. `select`, `where`, `orderBy`. 

```
testtable
  .select("name", "country")
  .where("name LIKE 'P%'")
  .orderBy("country"))
```
Transformations operate on and return DataFrames, allowing us to chain transformation methods together to construct new DataFrames.  
However, these operations **can't execute on their own**, as transformation methods are **lazily evaluated**. 

Running the following cell does not trigger any computation.

In [0]:
(usersP
  .select("name", "country")
  .where("name LIKE 'P%'")
  .orderBy("name"))

## Actions
Conversely, DataFrame actions are methods that **trigger computation**.  
Actions are needed to trigger the execution of any DataFrame transformations. 

For example, the `show()` action causes the following cell to execute transformations.

In [0]:
(usersP
  .select("name", "country")
  .where("name LIKE 'P%'")
  .orderBy("name")
  .show())

Below are several examples of <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#dataframe-apis" target="_blank">DataFrame</a> actions.

### DataFrame Action Methods
| Method | Description |
| --- | --- |
| show | Displays the top n rows of DataFrame in a tabular form |
| count | Returns the number of rows in the DataFrame |
| describe,  summary | Computes basic statistics for numeric and string columns |
| first, head | Returns the the first row |
| collect | Returns an array that contains all rows in this DataFrame |
| take | Returns an array of the first n rows in the DataFrame |

The action `count()` returns the number of records in a DataFrame.

In [0]:
usersP.count()

`collect()` returns an array of all rows in a DataFrame.

In [0]:
usersP.collect() 

## Convert between DataFrames and SQL

`createOrReplaceTempView` creates a temporary view based on the DataFrame. The lifetime of the temporary view is tied to the SparkSession that was used to create the DataFrame.

In [0]:
usersP.createOrReplaceTempView("usersPsqlview")

In [0]:
display(spark.sql("SELECT * FROM usersPsqlview"))

# Spark SQL Lab

##### Tasks
1. Create a DataFrame from the `testtable` table
1. Display the DataFrame and inspect its schema
1. Apply transformations to filter and sort
1. Count results and take the first 5 rows
1. Create the same DataFrame using a SQL query

##### Methods
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html?highlight=sparksession" target="_blank">SparkSession</a>: `sql`, `table`
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html" target="_blank">DataFrame</a> transformations: `select`, `where`, `orderBy`
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html" target="_blank">DataFrame</a> actions: `select`, `count`, `take`
- Other <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html" target="_blank">DataFrame</a> methods: `printSchema`, `schema`, `createOrReplaceTempView`

### 1. Create a DataFrame from the `testtable` table
- Use SparkSession to create a DataFrame from the `testtable` table

In [0]:
# ANSWER
users = spark.table("testtable")

### 2. Display DataFrame and inspect schema
- Use methods above to inspect DataFrame contents and schema

In [0]:
# ANSWER
display(users)

In [0]:
# ANSWER
users.printSchema()

### 3. Apply transformations to filter and sort
- Filter for rows where `country` is `Austria`
- Sort rows by `name`

<img src="https://files.training.databricks.com/images/icon_hint_32.png" alt="Hint"> Use single and double quotes in your filter SQL expression

In [0]:
# ANSWER
newusers = (users
            .where("country == 'Austria'")
            .sort("name")
           )

### 4. Count results and take first 5 rows
- Use DataFrame actions to count and take rows

In [0]:
# ANSWER
numRows = newusers.count()
print(numRows)

rows = newusers.take(5)
for row in rows:
  print(row)

### 5. Create the same DataFrame using SQL query
- Use SparkSession to run a SQL query on the `events` table
- Use SQL commands to write the same filter and sort the query used earlier

In [0]:
# ANSWER
usersSQL = spark.sql("""
SELECT *
FROM testtable
WHERE country = 'Austria'
ORDER By name
""")

display(usersSQL)